### Input table
##### Once a year
converged-havas-de.converged_user.natrep_info 
(only needed to compute the SEC natrep - once a year)
##### Once a week
converged-havas-de.converged_user.user_response_all_date
converged-havas-de.converged_user.weight_all_date
converged-havas-de.converged_user.codes_all_date

### Output table
##### Temp
converged-havas-de.converged_user.btr_panelist_profile_fr
##### Result of the script
converged-havas-de.converged_user.respondents_weights


## Import Library


In [1]:
install.packages("anesrake")
install.packages("bigrquery")
install.packages("data.table")
install.packages("googleCloudStorageR")
install.packages("bigQueryR")

Installing package into ‘/opt/conda/anaconda/lib/R/library’
(as ‘lib’ is unspecified)
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Installing package into ‘/opt/conda/anaconda/lib/R/library’
(as ‘lib’ is unspecified)
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Installing package into ‘/opt/conda/anaconda/lib/R/library’
(as ‘lib’ is unspecified)
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Installing package into ‘/opt/conda/anaconda/lib/R/library’
(as ‘lib’ is unspecified)
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Installing package into ‘/opt/conda/anaconda/lib/R/library’
(as ‘lib’ is unspecified)
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [2]:
library(bigrquery)
library(anesrake)
library("data.table")
library(googleCloudStorageR)
library(bigQueryR)

Loading required package: Hmisc
Loading required package: lattice
Loading required package: survival
Loading required package: Formula
Loading required package: ggplot2
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang

Attaching package: ‘Hmisc’

The following objects are masked from ‘package:base’:

    format.pval, units

Loading required package: weights


In [3]:
gcs_global_bucket("converged-havas-de")
billing <- "converged-havas-de" 

Set default bucket name to 'converged-havas-de'


In [4]:
bqr_auth(json_file = NULL, token = NULL, email = "converged-sa-user@converged-havas-de.iam.gserviceaccount.com")

ℹ 2021-06-04 09:38:39 > Setting client.id from options(googleAuthR.client_id)


In [5]:
projectId = "converged-havas-de"
datasetId = "converged_user"
tableId = "respondents_weight"

## Create the targets (nat rep) variables
### For France  will use : Age, Gender, SEC and Location
##### The scope will be 18 to 69 yo, for France Metropolitan. Because we will not use all the YouGov Data (older panelist will be deleted and panelist who did not answer one of the weighting variable will be deleted), we need to INNER JOIN the output of the script with the RLD data to make sure we keep only the panelist we want (approx 95/96% of the panelists)
##### So we will need something to check each week the % of panelist lost to monitore it.
##### 41295558. In 2021, there is 41295558 people in France who have between 18 and 69 yo in France Metropolitan.

In [6]:
total_pop = 41295558 

In [7]:
# Source : https://www.insee.fr/fr/statistiques/1893198
# File : modif FR demographics- Régions, genres, âges (2021-3)
# Scope : (18-69, France Metropolitain, 2021). 18 and 19 YO are restimated as 40% of the 15-19 category.
# RLD alias : pdl_gender
gender <- c(.51, .49)
names(gender) <- c("Female", "Male")
gender

Female   Male 
  0.51   0.49

In [8]:
# Source : https://www.insee.fr/fr/statistiques/1893198
# File : modif FR demographics- Régions, genres, âges (2021-3) 
# Scope : (18-69, France Metropolitain, 2021). 
# RLD alias : pdlc_agegrp
age <- c(.128, .181, .195, .206, .199, .091)
names(age) <- c("18-24", "25-34", "35-44", "45-54", "55-64", "65+")
age

18-24 25-34 35-44 45-54 55-64   65+ 
0.128 0.181 0.195 0.206 0.199 0.091

In [9]:
# Source : https://www.insee.fr/fr/statistiques/4995124?sommaire=2414232&q=Donn%C3%A9es+harmonis%C3%A9es+des+recensements#consulter
# File : [converged-havas-de:converged_user.natrep_info] Farmer added to Artisan, Retail & Entrepreneur to prevent low modality size
# Scope : (18-69, France Metropolitain, 2017). 
# RLD alias : pdl_Q_SEC
csp <- c(0.0538, 0.2037, 0.1178, 0.1458, 0.178, 0.1486, 0.1523)
names(csp) <- c("Artisan-Retail-Entrepreneur-Farmer", "Employee", "Executive and Intellectual profession", "Retired", "Intermediate occupation", "Other non-working", "Worker")
csp

Artisan-Retail-Entrepreneur-Farmer                              Employee 
                               0.0538                                0.2037 
Executive and Intellectual profession                               Retired 
                               0.1178                                0.1458 
              Intermediate occupation                     Other non-working 
                               0.1780                                0.1486 
                               Worker 
                               0.1523

In [10]:
# Source : https://www.insee.fr/fr/statistiques/1893198
# File : modif FR demographics- Régions, genres, âges (2021-3)
# Scope : (18-69, France Metropolitain, 2021). 18 and 19 YO are restimated as 40% of the 15-19 category.
# RLD alias : pdlc_NEW_regions_2015
location <- c(.088, .092, .124, .042, .051, .038, .197, 0.091, 0.092,0.05, 0.058, 0.077)
names(location) <- c("Alsace,Champagne-Ardenne,Lorraine", "Aquitaine, Limousin, Poitou-Charentes", "Auvergne, Rhône-Alpes", "Bourgogne, Franche-Comté", "Bretagne", "Centre", "Ile-de-France", "Languedoc-Roussillon, Midi-Pyrénées", "Nord-Pas-de-Calais, Picardie", "Normandie", "Pays de la Loire", "Provence-Alpes-Côte")
location

Alsace,Champagne-Ardenne,Lorraine Aquitaine, Limousin, Poitou-Charentes 
                                0.088                                 0.092 
                Auvergne, Rhône-Alpes              Bourgogne, Franche-Comté 
                                0.124                                 0.042 
                             Bretagne                                Centre 
                                0.051                                 0.038 
                        Ile-de-France   Languedoc-Roussillon, Midi-Pyrénées 
                                0.197                                 0.091 
         Nord-Pas-de-Calais, Picardie                             Normandie 
                                0.092                                 0.050 
                     Pays de la Loire                   Provence-Alpes-Côte 
                                0.058                                 0.077

In [11]:
targets <- list(age,gender,location,csp)
names(targets) <- c('age','gender','location','csp')

In [12]:
# Natrep Targets
targets

$age
18-24 25-34 35-44 45-54 55-64   65+ 
0.128 0.181 0.195 0.206 0.199 0.091 

$gender
Female   Male 
  0.51   0.49 

$location
    Alsace,Champagne-Ardenne,Lorraine Aquitaine, Limousin, Poitou-Charentes 
                                0.088                                 0.092 
                Auvergne, Rhône-Alpes              Bourgogne, Franche-Comté 
                                0.124                                 0.042 
                             Bretagne                                Centre 
                                0.051                                 0.038 
                        Ile-de-France   Languedoc-Roussillon, Midi-Pyrénées 
                                0.197                                 0.091 
         Nord-Pas-de-Calais, Picardie                             Normandie 
                                0.092                                 0.050 
                     Pays de la Loire                   Provence-Alpes-Côte 
                                0.058                                 0.077 

$csp
   Artisan-Retail-Entrepreneur-Farmer                              Employee 
                               0.0538                                0.2037 
Executive and Intellectual profession                               Retired 
                               0.1178                                0.1458 
              Intermediate occupation                     Other non-working 
                               0.1780                                0.1486 
                               Worker 
                               0.1523

## Creation of the dataset

In [13]:
sql <- "

CREATE OR REPLACE TABLE
  `converged-havas-de.converged_user.btr_panelist_profile_fr` AS (
  WITH
    sub AS (
    SELECT
      *
    FROM
      `converged-havas-de.converged_user.user_response_all_date`
    WHERE
      date=DATE(data.date)
      AND country_id = 7
      AND alias IN ('pdlc_agegrp',
        'pdlc_age',
        'pdl_gender',
        'pdlc_NEW_regions_2015',
        'pdl_Q_SEC') )
  SELECT
    weight.*,
    age.age,
    gender.gender,
    location.location,
    csp.csp
  FROM (
    SELECT
      country_id,
      date,
      yougov_id,
      CAST(response AS float64) AS weight
    FROM
      `converged-havas-de.converged_user.weight_all_date`
    WHERE
      CAST(date AS string) = data.date
      AND country_id = 7) AS weight
  INNER JOIN (
    SELECT
      country_id,
      date,
      yougov_id,
      name AS age
    FROM (
      SELECT
        response.*,
        codes.name
      FROM
        sub AS response
      INNER JOIN
        `converged-havas-de.converged_user.codes_all_date` AS codes
      ON
        response.country_id=codes.country_id
        AND response.alias = codes.definition_alias
        AND response.response = codes.value
      WHERE
        CAST(response.date AS string) = data.date
        AND response.country_id = 7
        AND response.alias = 'pdlc_agegrp'
        AND CAST(codes.date AS string) = data.date
        AND codes.country_id = 7
        AND codes.definition_alias = 'pdlc_agegrp')) AS age
  ON
    weight.country_id=age.country_id
    AND weight.date= age.date
    AND weight.yougov_id=age.yougov_id
  INNER JOIN (
    SELECT
      country_id,
      date,
      yougov_id
    FROM (
      SELECT
        response.*
      FROM
        sub AS response
      WHERE
        CAST(response.date AS string) = data.date
        AND response.country_id = 7
        AND response.alias = 'pdlc_age'
        AND CAST(response.response AS float64) >=18.0
        AND CAST(response.response AS float64) <70.0)) AS scope
  ON
    weight.country_id=scope.country_id
    AND weight.date= scope.date
    AND weight.yougov_id=scope.yougov_id
  INNER JOIN (
    SELECT
      country_id,
      date,
      yougov_id,
      name AS gender
    FROM (
      SELECT
        response.*,
        codes.name
      FROM
        sub AS response
      INNER JOIN
        `converged-havas-de.converged_user.codes_all_date` AS codes
      ON
        response.country_id=codes.country_id
        AND response.alias = codes.definition_alias
        AND response.response = codes.value
      WHERE
        CAST(response.date AS string) = data.date
        AND response.country_id = 7
        AND response.alias = 'pdl_gender'
        AND CAST(codes.date AS string) = data.date
        AND codes.country_id = 7
        AND codes.definition_alias = 'pdl_gender')) AS gender
  ON
    weight.country_id=gender.country_id
    AND weight.date= gender.date
    AND weight.yougov_id=gender.yougov_id
  INNER JOIN (
    SELECT
      country_id,
      date,
      yougov_id,
      name AS location
    FROM (
      SELECT
        response.*,
        codes.name
      FROM
        sub AS response
      INNER JOIN
        `converged-havas-de.converged_user.codes_all_date` AS codes
      ON
        response.country_id=codes.country_id
        AND response.alias = codes.definition_alias
        AND response.response = codes.value
      WHERE
        CAST(response.date AS string) = data.date
        AND response.country_id = 7
        AND response.alias IN ('pdlc_NEW_regions_2015')
        AND CAST(codes.date AS string) = data.date
        AND codes.country_id = 7
        AND codes.definition_alias IN ('pdlc_NEW_regions_2015') )) AS location
  ON
    weight.country_id=location.country_id
    AND weight.date= location.date
    AND weight.yougov_id=location.yougov_id
  INNER JOIN (
    SELECT
      country_id,
      date,
      yougov_id,
      name AS csp
    FROM (
      SELECT
        response.*,
        codes.name
      FROM
        sub AS response
      INNER JOIN
        `converged-havas-de.converged_user.codes_all_date` AS codes
      ON
        response.country_id=codes.country_id
        AND response.alias = codes.definition_alias
        AND response.response = codes.value
      WHERE
        CAST(response.date AS string) = data.date
        AND response.country_id = 7
        AND response.alias = 'pdl_Q_SEC'
        AND CAST(codes.date AS string) = data.date
        AND codes.country_id = 7
        AND codes.definition_alias = 'pdl_Q_SEC')) AS csp
  ON
    weight.country_id=csp.country_id
    AND weight.date= csp.date
    AND weight.yougov_id=csp.yougov_id )

"

In [14]:
th0 <- bq_project_query(billing, sql)
df0 <- bq_table_download(th0)

## Pre process the data

In [15]:
sql <- "

SELECT
  yougov_id,
  age,
  gender,
  location,
  (CASE
      WHEN csp IN('Artisan-Retail-Entrepreneur', 'Farmer')THEN 'Artisan-Retail-Entrepreneur-Farmer'
    ELSE
    csp
  END
    ) AS csp
FROM
  converged-havas-de.converged_user.btr_panelist_profile_fr
WHERE
  date = DATE(data.date)
  AND country_id = 7
  AND age IS NOT NULL
  AND gender IS NOT NULL
  AND location IS NOT NULL
  AND csp IS NOT NULL

"

In [16]:
th <- bq_project_query(billing, sql)
df <- bq_table_download(th)

## Prepare the ANES rank

In [17]:
df <- as.data.frame(df)

## Apply the ANES rank

In [18]:
df$yougov_id <- as.factor(df$yougov_id) 
df$age <- as.factor(df$age) 
df$gender <- as.factor(df$gender) 
df$location <- as.factor(df$location) 
df$csp <- as.factor(df$csp) 

In [19]:
outsave <- anesrake(targets, df, caseid=df$yougov_id, weightvec = NULL , 
cap = 5,
verbose = TRUE, maxit = 1000, type = "nolim", #all variables listed will be included in the weighting to make sure we will have the same weighting method between weeks
pctlim = 5, nlim = 5, filter = 1, choosemethod = "total",
iterate = TRUE, convcrit = 0.01, force1=TRUE, center.baseweights=TRUE)

#Good to know : Passek propose a function called anesrakefinder to find the variables we should used for weighting. For our case it is not helpful, we want to choose our variables.

[1] "Raking...Iteration 1"
[1] "Capping...Iteration 1.1"
[1] "Capping...Iteration 1.2"
[1] "Capping...Iteration 1.3"
[1] "Current iteration changed total weights by 72872.0961866228"
[1] "Raking...Iteration 2"
[1] "Capping...Iteration 2.1"
[1] "Current iteration changed total weights by 8583.6961270142"
[1] "Raking...Iteration 3"
[1] "Current iteration changed total weights by 1094.64589646411"
[1] "Raking...Iteration 4"
[1] "Current iteration changed total weights by 459.477702998629"
[1] "Raking...Iteration 5"
[1] "Current iteration changed total weights by 212.740347635988"
[1] "Raking...Iteration 6"
[1] "Current iteration changed total weights by 98.3172625378296"
[1] "Raking...Iteration 7"
[1] "Current iteration changed total weights by 45.3668225798492"
[1] "Raking...Iteration 8"
[1] "Current iteration changed total weights by 20.920351813245"
[1] "Raking...Iteration 9"
[1] "Current iteration changed total weights by 9.64444433981985"
[1] "Raking...Iteration 10"
[1] "Current iter

In [20]:
caseweights <- data.frame(cases=outsave$caseid, weights=outsave$weightvec)
head(caseweights)

,cases,weights
45JVIBPPFXEYI3GLUWBQXILQPI,45JVIBPPFXEYI3GLUWBQXILQPI,3.407399
N34OBU74M4CBIE757Y4WQ55NEI,N34OBU74M4CBIE757Y4WQ55NEI,1.180232
S6V4ZS5BJIAFGZPBORGDL7EIGY,S6V4ZS5BJIAFGZPBORGDL7EIGY,1.799251
NJOJ4UXYKSCF7JMQNFH7RCRIBQ,NJOJ4UXYKSCF7JMQNFH7RCRIBQ,1.799251
66BQSUG2Q3XEMXIOYYNGEPTIDE,66BQSUG2Q3XEMXIOYYNGEPTIDE,1.799251
BBGMPGMXE2COKGDFFBFTRAUZKI,BBGMPGMXE2COKGDFFBFTRAUZKI,1.799251


In [21]:
summary(outsave)

$convergence
[1] "Complete convergence was achieved after 46 iterations"

$base.weights
[1] "No Base Weights Were Used"

$raking.variables
[1] "age"      "gender"   "location" "csp"     

$weight.summary
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.3016  0.4257  0.7677  1.0000  1.2059  4.9720 

$selection.method
[1] "variable selection conducted using _nolim_ - discrepancies selected using _total_."

$general.design.effect
[1] 1.686021

$age
      Target Unweighted N Unweighted %     Wtd N Wtd % Change in % Resid. Disc.
18-24  0.128        18153   0.15165287  15321.73 0.128 -0.02365287 0.000000e+00
25-34  0.181        29109   0.24318093  21665.88 0.181 -0.06218093 2.775558e-17
35-44  0.195        29179   0.24376572  23341.70 0.195 -0.04876572 2.775558e-17
45-54  0.206        22064   0.18432595  24658.41 0.206  0.02167405 0.000000e+00
55-64  0.199        15830   0.13224618  23820.50 0.199  0.06675382 0.000000e+00
65+    0.091         5366   0.04482836  10892.79 0.091  0.04617164 

In [22]:
caseweights$country_id <- '7'
caseweights$date <- '2022-01-06'

In [23]:
names(caseweights)[names(caseweights) == 'cases'] <- 'yougov_id'

In [24]:
names(caseweights)[names(caseweights) == 'weights'] <- 'weight'

In [25]:
data_all = caseweights
head(data_all)

,yougov_id,weight,country_id,date
45JVIBPPFXEYI3GLUWBQXILQPI,45JVIBPPFXEYI3GLUWBQXILQPI,3.407399,7,2021-04-18
N34OBU74M4CBIE757Y4WQ55NEI,N34OBU74M4CBIE757Y4WQ55NEI,1.180232,7,2021-04-18
S6V4ZS5BJIAFGZPBORGDL7EIGY,S6V4ZS5BJIAFGZPBORGDL7EIGY,1.799251,7,2021-04-18
NJOJ4UXYKSCF7JMQNFH7RCRIBQ,NJOJ4UXYKSCF7JMQNFH7RCRIBQ,1.799251,7,2021-04-18
66BQSUG2Q3XEMXIOYYNGEPTIDE,66BQSUG2Q3XEMXIOYYNGEPTIDE,1.799251,7,2021-04-18
BBGMPGMXE2COKGDFFBFTRAUZKI,BBGMPGMXE2COKGDFFBFTRAUZKI,1.799251,7,2021-04-18


# Export the new weight for FR in BQ

In [26]:
## custom upload function to ignore quotes and column headers
f <- function(input, output) {
write.table(input, sep = ",", col.names = FALSE, row.names = FALSE, 
           quote = FALSE, file = output, qmethod = "double")}

In [27]:
## create the schema of the files you just uploaded
user_schema <- schema_fields(data_all)

In [28]:
bqr_upload_data(projectId = projectId, 
                datasetId = datasetId, 
                tableId = tableId, 
                upload_data = data_all,
                schema = user_schema)

2021-06-04 09:39:05 -- Uploading local data.frame
2021-06-04 09:39:07 -- Upload job made...
2021-06-04 09:39:12 -- Waiting for job: job_-YzQ0UtQ6Dy73S4WCqTzbxMvkN97 - Job timer: 5.001389 secs
ℹ 2021-06-04 09:39:12 > Request Status Code:  404


ERROR: Error: API returned: Not found: Job converged-havas-de:job_-YzQ0UtQ6Dy73S4WCqTzbxMvkN97
